Create DataFrames for each patient to train SVM

In [1]:
%matplotlib inline
import pandas as pd
import SimpleITK as sitk
import glob
import os
import re
import matplotlib.pyplot as plt

Discrete DataFrame creation

In [6]:
# function to create DataFrame columns using lesion info
def create_col(dir, p_ID, mode):
    # modalities =  GM, MD, FA, FICVF, ODI
    modes = {'GM': dir + 'GM-LESIONS/',
            'MD': dir + 'MD-LESIONS/',
            'FA': dir + 'FA-LESIONS/',
            'FICVF': dir + 'FICVF-LESIONS/',
            'ODI': dir + 'ODI-LESIONS/'
           }
    # data is sitk image type
    # p_ID is patient ID (string)
    
    file = glob.glob( modes[mode] + "*" + p_ID + "*.nii")
    if not file:
        print(mode + " modality doesn't exist for " + p_ID)
        return [0] * 2122945 # voxels in flattened mri

    img = sitk.ReadImage(file[0])
    arr = sitk.GetArrayFromImage(img)
    arr = arr.flatten()
    return arr
   
gm_lesions_file_list = glob.glob("OUTPUT/DISCRETE/LESIONS/GM-LESIONS/*")
gm_lesions_file_list.sort()
    
patient_ids = []

for file in gm_lesions_file_list:
    patient_ids.append(re.search('.*(D[0-9][0-9][0-9]).*', os.path.basename(file)).group(1))

ALL_D_FRAMES =[]

# create discrete df
print("Creating discrete DataFrame...")
dir = "OUTPUT/DISCRETE/LESIONS/"

for patient in patient_ids:
    df = pd.DataFrame({'GM' : create_col(dir, patient, "GM"),
                        'FA' : create_col(dir, patient, "FA"),
                        'MD' : create_col(dir, patient, "MD"),
                        'FICVF' : create_col(dir, patient, "FICVF"),
                        'ODI' : create_col(dir, patient, "ODI") ,
                      
    })
    
    df = df[(df.T != 0).any()]
    df['target_name'] = "Discrete"
    df['target'] = 1
    df = df.reset_index(drop=True)    
    ALL_D_FRAMES.append(df)


# sizes = []
# for i in range(len(ALL_D_FRAMES)):
#     sizes.append(ALL_D_FRAMES[i].shape[0])
# tot = 0
# for i in range(len(sizes)):
#     tot = tot + sizes[i]
# print(tot)
#
# #check mean and range of sizes of lesions using boxplot
# df = pd.DataFrame(sizes, columns=['lesion_sizes'])
# boxplot = df.boxplot()
# plt.savefig("boxplot-image")

# remove outliers from analysis, will be tested on later
outlier_frames = []
outliers = [32, 31, 22, 19, 17, 11] # outlier indices, reverse order not to mess up indexing
for i in outliers:
    outlier_frames.append(ALL_D_FRAMES[i])
    ALL_D_FRAMES.pop(i)

print("Discrete DataFrame created.")

Creating discrete DataFrame...
Discrete DataFrame created.


Control DataFrame creation

In [7]:
# get patient IDs
gm_lsn_files = glob.glob("OUTPUT/CONTROL/LESIONS/GM-LESIONS/*")
gm_lsn_files.sort()
patient_ids = []
for file in gm_lsn_files:
    patient_ids.append(re.search('.*(C[0-9][0-9][0-9]).*', os.path.basename(file)).group(1))

# create control df
print("Creating control DataFrame...")
dir = "OUTPUT/CONTROL/LESIONS/"
ALL_C_FRAMES = []
for patient in patient_ids:
    data = {'GM' : create_col(dir, patient, "GM"),
                        'FA' : create_col(dir, patient, "FA"),
                        'MD' : create_col(dir, patient, "MD"),
                        'FICVF' : create_col(dir, patient, "FICVF"),
                        'ODI' : create_col(dir, patient, "ODI"),      
    }
    df = pd.DataFrame(data, index=None)
    df = df[(df.T != 0).any()]
    df['target_name'] = "Control"
    df['target'] = 0
    df = df.reset_index(drop=True)    
    ALL_C_FRAMES.append(df)

## make sure there aren't any outliers in the control group
# control_sizes = []
# for i in range(len(ALL_C_FRAMES)):
#     control_sizes.append(ALL_C_FRAMES[i].shape[0])
# tot = 0
# for i in range(len(control_sizes)):
#     tot = tot + control_sizes[i]
# print(tot)
#
# df2 = pd.DataFrame(control_sizes, columns=['lesion_sizes'])
# boxplot = df2.boxplot()
# plt.savefig("boxplot-image-no-outliers-CONTROL")

print("Control DataFrame created.")

Creating control DataFrame...
Control DataFrame created.


In [5]:
# Although we are using less Control patients, the number of voxels is still balanced 
# between Control and Discrete, which is important for the training step
print("Creating train and test DataFrames...")
c_train = ALL_C_FRAMES[0:26] # get 25 controls for train
d_train = ALL_D_FRAMES[0:28] # get 27 discrete for train
c_train_df = pd.concat(c_train)
d_train_df = pd.concat(d_train)
train_df = c_train_df.append(d_train_df)
train_df = train_df.reset_index(drop=True)

c_test = ALL_C_FRAMES[45:64] # get 18 controls for test
d_test = ALL_D_FRAMES[28:38] # get 11 discrete for test

c_test_df = pd.concat(c_test)
d_test_df = pd.concat(d_test)
test_df = c_test_df.append(d_test_df)
test_df = test_df.reset_index(drop=True)
print("Train and test DataFrames created.")

print("Writing train and test data to csv files...")
train_df.to_csv('train-data.csv')
test_df.to_csv('test-data.csv')
print("Train and test data files saved.")

## We want to ensure that there are is a balance of discrete and control voxels
## in the training phase to have balanced results
# print(c_train_df.shape[0])
# print(d_train_df.shape[0])
# print(c_test_df.shape[0])
# print(d_test_df.shape[0])


Creating train and test DataFrames...
Train and test DataFrames created.
Writing train and test data to csv files...
Train and test data files saved.
